In [ ]:
import logging
import time
import xarray as xr
import glob
import pandas as pd

from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
from examples.download_kernels import download_kernels
from src.config import ShSpOpt
from src.shadowspy.data_handling import fetch_and_process_data
from src.shadowspy.dem_processing import prepare_dem_mesh
from src.shadowspy.helpers import setup_directories, process_data_list
from src.shadowspy.raster_products import basic_raster_stats
from src.shadowspy.utilities import run_log

Setup config and directories

In [ ]:
opt = ShSpOpt()
opt.setup_config(config_file='MoonSP_render_config.yaml')
opt.set(siteid='MoonSPsmall')
opt.display()

setup_directories(opt)

download kernels

In [ ]:
if opt.download_kernels:
    download_kernels()

prepare mesh of the input dem

In [ ]:
start = time.time()
logging.info(f"- Computing trimesh for {opt.dem_path}...")
inner_mesh_path, outer_mesh_path, dem_path = prepare_dem_mesh(opt.dem_path, opt.tmpdir, opt.siteid, opt)
logging.info(f"- Meshes generated after {round(time.time() - start, 2)} seconds.")

Determine the mode and prepare data list

In [ ]:
data_list, use_azi_ele, use_image_times = fetch_and_process_data(opt)
print(f"- Illuminating input DEM at {data_list}.")

Common arguments for both cases

In [ ]:
common_args = {
    'meshes': {'stereo': f"{inner_mesh_path}_st{opt.mesh_ext}", 'cart': f"{inner_mesh_path}{opt.mesh_ext}"},
    'basemesh_path': outer_mesh_path + opt.mesh_ext,
    'path_to_furnsh': f"{opt.indir}simple.furnsh",
    'point': opt.point_source,
    'extsource_coord': opt.extsource_coord,
    'source': opt.source,
    'dem_path': dem_path,
    'crs': 'PROJCS["Moon (2015) - Sphere / Ocentric / South Polar", BASEGEOGCRS["Moon (2015) - Sphere / Ocentric", DATUM["Moon (2015) - Sphere", ELLIPSOID["Moon (2015) - Sphere",1737400,0, LENGTHUNIT["metre",1]]], PRIMEM["Reference Meridian",0, ANGLEUNIT["degree",0.0174532925199433]], ID["IAU",30100,2015]], CONVERSION["South Polar", METHOD["Polar Stereographic (variant A)", ID["EPSG",9810]], PARAMETER["Latitude of natural origin",-90, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8801]], PARAMETER["Longitude of natural origin",0, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8802]], PARAMETER["Scale factor at natural origin",1, SCALEUNIT["unity",1], ID["EPSG",8805]], PARAMETER["False easting",0, LENGTHUNIT["metre",1], ID["EPSG",8806]], PARAMETER["False northing",0, LENGTHUNIT["metre",1], ID["EPSG",8807]]], CS[Cartesian,2], AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1]], AXIS["(N)",north, ORDER[2], LENGTHUNIT["metre",1]], ID["IAU",30135,2015]]'
}

actually compute irradiance at each element of data_list

In [ ]:
dsi_epo_path_dict = process_data_list(data_list, common_args, use_azi_ele, use_image_times, opt)

In [ ]:
render_paths = glob.glob(f"{opt.outdir}{opt.siteid}/{opt.siteid}_*.tif")[:]
pd.DataFrame.from_records([dsi_epo_path_dict]).T.to_csv(f'{opt.outdir}{opt.siteid}/dsi_epo_paths.csv')

fig, axes = plt.subplots(nrows=len(render_paths), ncols=1)
for idx, render in tqdm(enumerate(render_paths)):
    da = xr.open_dataarray(render)
    # da = da.sel(band=1).coarsen({'x': 3, 'y': 3}, boundary='trim').mean() 
    da.plot(robust=True, cmap='Greys', ax=axes[idx])

plt.show()

prepare mean, sum, max stats rasters and set-up log file

In [ ]:
# from datetime import datetime
dsi_epo_path = pd.read_csv(f'{opt.outdir}{opt.siteid}/dsi_epo_paths.csv').set_index('Unnamed: 0').to_dict()['0']
# dsi_epo_path = {datetime.strptime(str(k), '%Y%m%d%H%M%S'):v for k,v in dsi_epo_path.items()}
dem = xr.open_dataarray(common_args['dem_path'])
basic_raster_stats(dsi_epo_path, opt.time_step_hours, crs=dem.rio.crs, outdir=opt.outdir, siteid=opt.siteid)

In [ ]:
run_log(Fsun=opt.Fsun, Rb=opt.Rb, base_resolution=opt.base_resolution, siteid=opt.siteid, dem_path=dem_path, outdir=opt.outdir,
        start_time=opt.start_time, end_time=opt.end_time, time_step_hours=opt.time_step_hours,
        runtime_sec=round(time.time() - start, 2), logpath=f"{opt.outdir}{opt.siteid}/illum_stats_{opt.siteid}_{int(time.time())}.json")
logging.info(f"Completed in {round(time.time() - start, 2)} seconds.")